In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

# Dataförberedelse
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Skal om CIFAR-10 bilder från 32x32 till 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

def train_model(model, criterion, optimizer, train_loader, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)    

            if (i + 1) % 100 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct_train / total_train:.2f}%")
    
    torch.save(model.state_dict(), "best_model.pth")

def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


model_ft = models.alexnet(pretrained=True)

for param in model_ft.features.parameters():
    param.requires_grad = False

num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, 10)  

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.classifier[6].parameters(), lr=0.001)

train_model(model_ft, criterion, optimizer, trainloader, num_epochs=10)

model_ft.load_state_dict(torch.load("best_model.pth"))
test_model(model_ft, testloader)



Files already downloaded and verified
Files already downloaded and verified


C:\Users\lindh\anaconda3\envs\nnlm\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lindh\anaconda3\envs\nnlm\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Batch 100/782, Loss: 0.9995
Epoch 1/10, Batch 200/782, Loss: 0.9664
Epoch 1/10, Batch 300/782, Loss: 0.8668
Epoch 1/10, Batch 400/782, Loss: 0.6397
Epoch 1/10, Batch 500/782, Loss: 0.8640
Epoch 1/10, Batch 600/782, Loss: 0.9692
Epoch 1/10, Batch 700/782, Loss: 0.5854
Epoch 1/10, Average Loss: 0.8795, Accuracy: 69.08%
Epoch 2/10, Batch 100/782, Loss: 0.5480
Epoch 2/10, Batch 200/782, Loss: 0.6920
Epoch 2/10, Batch 300/782, Loss: 0.7540
Epoch 2/10, Batch 400/782, Loss: 0.7450
Epoch 2/10, Batch 500/782, Loss: 0.5031
Epoch 2/10, Batch 600/782, Loss: 0.5121
Epoch 2/10, Batch 700/782, Loss: 0.6035
Epoch 2/10, Average Loss: 0.7810, Accuracy: 72.48%
Epoch 3/10, Batch 100/782, Loss: 0.6766
Epoch 3/10, Batch 200/782, Loss: 0.7538
Epoch 3/10, Batch 300/782, Loss: 0.4985
Epoch 3/10, Batch 400/782, Loss: 0.8978
Epoch 3/10, Batch 500/782, Loss: 0.7691
Epoch 3/10, Batch 600/782, Loss: 0.8893
Epoch 3/10, Batch 700/782, Loss: 0.9139
Epoch 3/10, Average Loss: 0.7602, Accuracy: 73.18%
Epoch 4

C:\Users\lindh\AppData\Local\Temp\ipykernel_2336\1341373466.py:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_ft.load_state_dict(torch.load("best_model.pth"))


Test Accuracy: 77.37%
